
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# File location and type
file_location = "/FileStore/tables/suv_data.csv"
file_type = "csv"

# CSV options: set header to 'true' since your file contains headers
suv_car_df = spark.read.format(file_type) \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

# Convert the DataFrame to Pandas for easier processing with scikit-learn
suv_car_pd = suv_car_df.toPandas()

# Strip any leading/trailing spaces from the column names
suv_car_pd.columns = suv_car_pd.columns.str.strip()

# Check if 'Purchased' exists in the dataset
if 'Purchased' in suv_car_pd.columns:
    print("'Purchased' column exists.")
else:
    print("'Purchased' column not found.")

# Display the column names to verify
print(suv_car_pd.columns)


'Purchased' column exists.
Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')


In [0]:
# Create a view or table

suv_car_pd.columns = suv_car_pd.columns.str.strip()

In [0]:
# Check if 'Purchased' exists in the dataset
if 'Purchased' in suv_car_pd.columns:
    print("'Purchased' column exists.")
else:
    print("'Purchased' column not found.")

# Display the column names to verify
print(suv_car_pd.columns)

'Purchased' column exists.
Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')


In [0]:
# Load the dataset and inspect its structure
suv_car_pd = suv_car_df.toPandas()

# Check the column names in the dataset
suv_car_pd.columns

# Check for missing values in the dataset
suv_car_pd.isnull().sum()

# Assuming 'Price' is the target variable and other columns are features
# Fill missing values with the median for numeric features
suv_car_pd.fillna(suv_car_pd.median(), inplace=True)

# Check the categorical columns in the dataset and encode them
categorical_columns = suv_car_pd.select_dtypes(include=['object']).columns

# Convert categorical features to numeric using Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Apply label encoding to all categorical columns
for col in categorical_columns:
    suv_car_pd[col] = label_encoder.fit_transform(suv_car_pd[col])

# Split data into features and target variable
X = suv_car_pd.drop(columns=['Purchased'])  # Assuming 'Price' is the target variable
y = suv_car_pd['Purchased']

# Display the modified dataset
suv_car_pd.head()


<command-2373370550541103>:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  suv_car_pd.fillna(suv_car_pd.median(), inplace=True)


,User ID,Gender,Age,EstimatedSalary,Purchased
0,97,1,1,43,0
1,392,1,17,44,0
2,155,0,8,65,0
3,64,0,9,78,0
4,382,1,1,96,0


In [0]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


Training set size: 320
Test set size: 80


In [0]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)


Out[10]: RandomForestRegressor(random_state=42)

In [0]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.07148
R-squared: 0.6858021978021979


In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model instantiation
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 0.09072143551886436
R-squared: 0.6012244592577392


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [0]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

grid_search = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")


Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}


In [0]:
# Using the best parameters found by GridSearchCV
best_model = GradientBoostingRegressor(
    n_estimators=50, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

# Train the model
best_model.fit(X_train, y_train)

# Predictions
y_pred_best = best_model.predict(X_test)

# Model evaluation
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Mean Squared Error: {mse_best}")
print(f"R-squared: {r2_best}")


Mean Squared Error: 0.08353264027673692
R-squared: 0.6328235592231345


In [0]:
pip install xgboost


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# File location and type
file_location = "/FileStore/tables/suv_data.csv"
file_type = "csv"

# CSV options: set header to 'true' since your file contains headers
suv_car_df = spark.read.format(file_type) \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

# Convert the DataFrame to Pandas for easier processing with scikit-learn
suv_car_pd = suv_car_df.toPandas()

# Strip any leading/trailing spaces from the column names
suv_car_pd.columns = suv_car_pd.columns.str.strip()

# Check if 'Purchased' exists in the dataset
if 'Purchased' in suv_car_pd.columns:
    print("'Purchased' column exists.")
else:
    print("'Purchased' column not found.")

# Display the column names to verify
print(suv_car_pd.columns)

'Purchased' column exists.
Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')


In [0]:
# Load the dataset and inspect its structure
suv_car_pd = suv_car_df.toPandas()

# Check the column names in the dataset
suv_car_pd.columns

# Check for missing values in the dataset
suv_car_pd.isnull().sum()

# Assuming 'Price' is the target variable and other columns are features
# Fill missing values with the median for numeric features
suv_car_pd.fillna(suv_car_pd.median(), inplace=True)

# Check the categorical columns in the dataset and encode them
categorical_columns = suv_car_pd.select_dtypes(include=['object']).columns

# Convert categorical features to numeric using Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Apply label encoding to all categorical columns
for col in categorical_columns:
    suv_car_pd[col] = label_encoder.fit_transform(suv_car_pd[col])

# Split data into features and target variable
X = suv_car_pd.drop(columns=['Purchased'])  # Assuming 'Price' is the target variable
y = suv_car_pd['Purchased']

# Display the modified dataset
suv_car_pd.head()

<command-3592271566191607>:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  suv_car_pd.fillna(suv_car_pd.median(), inplace=True)


,User ID,Gender,Age,EstimatedSalary,Purchased
0,97,1,1,43,0
1,392,1,17,44,0
2,155,0,8,65,0
3,64,0,9,78,0
4,382,1,1,96,0


In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the XGBoost model
model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_xgb.fit(X_train, y_train)

# Predictions
y_pred_xgb = model_xgb.predict(X_test)

# Model evaluation
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost MSE: {mse_xgb}")
print(f"XGBoost R²: {r2_xgb}")


XGBoost MSE: 0.08147501836871768
XGBoost R²: 0.6418680511265158


In [0]:
pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# File location and type
file_location = "/FileStore/tables/suv_data.csv"
file_type = "csv"

# CSV options: set header to 'true' since your file contains headers
suv_car_df = spark.read.format(file_type) \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

# Convert the DataFrame to Pandas for easier processing with scikit-learn
suv_car_pd = suv_car_df.toPandas()

# Strip any leading/trailing spaces from the column names
suv_car_pd.columns = suv_car_pd.columns.str.strip()

# Check if 'Purchased' exists in the dataset
if 'Purchased' in suv_car_pd.columns:
    print("'Purchased' column exists.")
else:
    print("'Purchased' column not found.")

# Display the column names to verify
print(suv_car_pd.columns)

'Purchased' column exists.
Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')


In [0]:
# Load the dataset and inspect its structure
suv_car_pd = suv_car_df.toPandas()

# Check the column names in the dataset
suv_car_pd.columns

# Check for missing values in the dataset
suv_car_pd.isnull().sum()

# Assuming 'Price' is the target variable and other columns are features
# Fill missing values with the median for numeric features
suv_car_pd.fillna(suv_car_pd.median(), inplace=True)

# Check the categorical columns in the dataset and encode them
categorical_columns = suv_car_pd.select_dtypes(include=['object']).columns

# Convert categorical features to numeric using Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Apply label encoding to all categorical columns
for col in categorical_columns:
    suv_car_pd[col] = label_encoder.fit_transform(suv_car_pd[col])

# Split data into features and target variable
X = suv_car_pd.drop(columns=['Purchased'])  # Assuming 'Price' is the target variable
y = suv_car_pd['Purchased']

# Display the modified dataset
suv_car_pd.head()

<command-3592271566191611>:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  suv_car_pd.fillna(suv_car_pd.median(), inplace=True)


,User ID,Gender,Age,EstimatedSalary,Purchased
0,97,1,1,43,0
1,392,1,17,44,0
2,155,0,8,65,0
3,64,0,9,78,0
4,382,1,1,96,0


In [0]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and fit the LightGBM model
model_lgb = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_lgb.fit(X_train, y_train)

# Predictions
y_pred_lgb = model_lgb.predict(X_test)

# Model evaluation
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)

print(f"LightGBM MSE: {mse_lgb}")
print(f"LightGBM R²: {r2_lgb}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 221
[LightGBM] [Info] Number of data points in the train set: 320, number of used features: 4
[LightGBM] [Info] Start training from score 0.359375
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [0]:
from sklearn.svm import SVR

# Initialize and fit the SVR model
model_svr = SVR(kernel='rbf', C=100, epsilon=0.1)
model_svr.fit(X_train, y_train)

# Predictions
y_pred_svr = model_svr.predict(X_test)

# Model evaluation
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print(f"SVR MSE: {mse_svr}")
print(f"SVR R²: {r2_svr}")

SVR MSE: 0.0900799269570916
SVR R²: 0.6040442771116853


In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

best_model = GradientBoostingRegressor(
    n_estimators=50, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

# Use cross-validation on the GradientBoostingRegressor
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')

print(f"Cross-validated MSE: {-cv_scores.mean()}")

Cross-validated MSE: 0.0924363212792697


In [0]:
import lightgbm as lgb

# Initialize and fit the LightGBM model
model_lgb = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_lgb.fit(X_train, y_train)

# Predictions
y_pred_lgb = model_lgb.predict(X_test)

# Model evaluation
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)

print(f"LightGBM MSE: {mse_lgb}")
print(f"LightGBM R²: {r2_lgb}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 221
[LightGBM] [Info] Number of data points in the train set: 320, number of used features: 4
[LightGBM] [Info] Start training from score 0.359375
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize and fit the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Predictions
y_pred_log_reg = log_reg.predict(X_test)

# Model evaluation
mse_log_reg = mean_squared_error(y_test, y_pred_log_reg)
r2_log_reg = r2_score(y_test, y_pred_log_reg)

print(f"Logistic Regression MSE: {mse_log_reg}")
print(f"Logistic Regression R²: {r2_log_reg}")


Logistic Regression MSE: 0.0875
Logistic Regression R²: 0.6153846153846154
